# Initial AI Agent PPO w CNN (working) MR2 HS Disc

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 4.0
my_config["ep_max_length"] = 2000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 7,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 1 is High Speed Ring, 2 is 0-400m
}

In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config : env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-07 23:40:48,996	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8266


In [5]:
import GTCustomNet
from ray.rllib.models.catalog import ModelCatalog
ModelCatalog.register_custom_model("complex_model", GTCustomNet.GTCustomNet)

2023-10-07 23:40:51,745	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [6]:
from ray.rllib.algorithms.ppo import PPOConfig
config = (
    PPOConfig()
    .resources(
        num_gpus=1,
        num_gpus_per_learner_worker=1,
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=128,
        model={
            "custom_model": "GTCustomNet.GTCustomNet",
            "custom_model_config": {},
            },
        _enable_learner_api=False
        )
    .rl_module( _enable_rl_module_api=False)
)

#PPOConfig.framework("")

2023-10-07 23:40:52,416	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-10-07 23:40:52,416	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.


In [7]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        try:
            json.dumps(obj)
        except TypeError:
            return "Not serializable object!"

        return obj

config_dict = config.to_dict()

print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

{
    "_AlgorithmConfig__prior_exploration_config": null,
    "_disable_action_flattening": false,
    "_disable_execution_plan_api": true,
    "_disable_initialize_loss_from_dummy_batch": false,
    "_disable_preprocessor_api": false,
    "_enable_learner_api": false,
    "_enable_rl_module_api": false,
    "_fake_gpus": false,
    "_is_atari": null,
    "_learner_class": null,
    "_tf_policy_handles_more_than_one_loss": false,
    "action_mask_key": "action_mask",
    "action_space": null,
    "actions_in_input_normalized": false,
    "algorithm_config_overrides_per_module": {},
    "always_attach_evaluation_results": false,
    "auto_wrap_old_gym_envs": true,
    "batch_mode": "truncate_episodes",
    "callbacks": "Not serializable object!",
    "checkpoint_trainable_policies_only": false,
    "clip_actions": false,
    "clip_param": 0.3,
    "clip_rewards": null,
    "compress_observations": false,
    "count_steps_by": "env_steps",
    "create_env_on_driver": false,
    "custom_e

In [8]:
algo = config.build()

2023-10-07 23:40:52,486	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\rllib\algorithms\algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in fut

(RolloutWorker pid=11764) GT Real Time instantiated
(RolloutWorker pid=11764) GT AI Server instantiated for rtgym
(RolloutWorker pid=11764) starting up on localhost port 9999
(RolloutWorker pid=11764) Waiting for a connection


2023-10-07 23:41:03,779	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=11764, ip=127.0.0.1, actor_id=522a04950a9926280c84b75301000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001E530A6CB80>)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\rllib\evaluation\rollout_worker.py",

(RolloutWorker pid=11764) Connection from ('127.0.0.1', 61035)
(RolloutWorker pid=11764) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2
(RolloutWorker pid=11764) I found cov2d layers


(RolloutWorker pid=11764) 2023-10-07 23:41:03,732	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(RolloutWorker pid=11764) 2023-10-07 23:41:03,734	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=11764) 2023-10-07 23:41:03,735	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=11764) 2023-10-07 23:41:

ValueError: Expected flattened obs shape of [..., 25316], got torch.Size([32, 5305])

In [ ]:
algo.get_policy().model

In [ ]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()
